In [1]:
#basic tools 
import os
import numpy as np
import pandas as pd
import pickle

#tuning hyperparameters
from bayes_opt import BayesianOptimization  

#building models
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
df_train_validation = pd.read_csv("final_train_val.csv", low_memory=False, index_col="id")
df_test = pd.read_csv("final_test.csv", low_memory=False, index_col="id")

/home/rico/anaconda3/envs/flight/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df_train_validation = reduce_mem_usage(df_train_validation) 
df_test = reduce_mem_usage(df_test) 

Mem. usage decreased to 174.72 Mb (73.8% reduction)
Mem. usage decreased to 30.90 Mb (73.8% reduction)


In [5]:
df_train_validation

,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,LATITUDE_origin,LONGITUDE_origin,...,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,scaled_DEPARTURE_TIME,scaled_WHEELS_OFF,scaled_SCHEDULED_TIME,scaled_DISTANCE
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,4,98,5,21.0,430,-22.0,61.187500,-150.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.251953,-1.628906,0.921387,1.161133
1,1,1,4,2336,10,12.0,750,-9.0,33.937500,-118.4375,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.626953,-1.629883,1.763672,2.443359
2,1,1,4,840,20,16.0,806,5.0,37.625000,-122.3750,...,0.0,0.0,0.0,1.0,0.0,0.0,-1.607422,-1.605469,1.831055,2.394531
3,1,1,4,258,20,15.0,805,-9.0,33.937500,-118.4375,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.610352,-1.610352,1.820312,2.460938
4,1,1,4,135,25,11.0,320,-21.0,47.437500,-122.3125,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.599609,-1.604492,1.258789,1.161133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,6,30,2,1180,2359,11.0,600,-8.0,33.937500,-118.4375,...,0.0,0.0,1.0,0.0,0.0,0.0,1.256836,-1.635742,1.326172,1.591797
2818549,6,30,2,1192,2359,11.0,520,79.0,33.937500,-118.4375,...,0.0,0.0,1.0,0.0,0.0,0.0,-1.471680,-1.475586,0.876465,1.061523
2818550,6,30,2,1480,2359,8.0,608,107.0,45.593750,-122.6250,...,0.0,0.0,1.0,0.0,0.0,0.0,-1.443359,-1.402344,1.416016,1.708984


In [6]:
# Split: for LGBM
df_train, df_validation = train_test_split(df_train_validation, test_size=0.20, random_state = 42)
X_train, y_train = df_train.drop("ARRIVAL_DELAY", axis=1), df_train["ARRIVAL_DELAY"]
X_val, y_val = df_validation.drop("ARRIVAL_DELAY", axis=1), df_validation["ARRIVAL_DELAY"]
X_test = df_test

In [13]:
def bayes_parameter_opt_lgbm(X, y, init_round=15, opt_round=25, n_folds=3, random_seed=6, output_process=False):
    # prepare data
    train_data = lgbm.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgbm_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf, min_sum_hessian_in_leaf, subsample):
        params = {'application':'regression_l2', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params['num_leaves'] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgbm.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval=200, metrics=['auc'])
        return max(cv_result['auc-mean'])
     
    lgbmBO = BayesianOptimization(lgbm_eval, {'learning_rate': (0.001, 1.0),
                                            'num_leaves': (2, 2**10),
                                            'feature_fraction': (0.1, 1),
                                            'bagging_fraction': (0.1, 1),
                                            'max_depth': (2, 10),
                                            'max_bin':(10,200),
                                            'min_data_in_leaf': (10, 200),
                                            'min_sum_hessian_in_leaf':(0,400),
                                            'subsample': (0.01, 1.0)}, random_state=200)

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbmBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_auc=[]
    for model in range(len( lgbmBO.res)):
        model_auc.append(lgbmBO.res[model]['target'])
    
    # return best parameters
    return lgbmBO.res[pd.Series(model_auc).idxmax()]['target'],lgbmBO.res[pd.Series(model_auc).idxmax()]['params']

In [14]:
opt_params = bayes_parameter_opt_lgbm(X_train, y_train, init_round=5, opt_round=10, n_folds=3, random_seed=6)
print(opt_params)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032


/home/rico/anaconda3/envs/flight/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045535 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 1503228, number of used features: 29
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] bagging_fraction is set=0.952869031166

/home/rico/anaconda3/envs/flight/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Start training from score 5.988886
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
|  1        |  0.8864   |  0.9529   |  0.2539   |  2.973    |  91.38    |  76.89    |  10.54    |  143.0    |  364.1    |  0.4615   |
[LightGBM] [Warning] bagging_fraction is set=0.9528690311665032, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9528690311665032
[LightGBM] [Warning] bagging_fraction is set=0.9836224382424957, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.9836224382424957
[LightGBM] [Warning] bagging_fraction is set=0.9836224382424957, subsample=0.2580000042232922 will be ignored. Current value: bagging_fraction=0.9836224382424957


...

[LightGBM] [Fatal] Cannot change max_bin after constructed Dataset handle.
/home/rico/anaconda3/envs/flight/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=0.01 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=0.01 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.061865 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1264
[LightGBM] [Info] Number of data points in the train set: 1503228, number of used features: 29
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=0.01 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=0.01 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=0.01 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011393 seconds.
You can s

/home/rico/anaconda3/envs/flight/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Start training from score 5.989361
[LightGBM] [Info] Start training from score 5.988886


KeyboardInterrupt: 

In [10]:
opt_params

(0.8962938098165422,
 {'bagging_fraction': 0.8,
  'feature_fraction': 0.95,
  'learning_rate': 1.0,
  'max_bin': 90.0,
  'max_depth': 30.0,
  'min_data_in_leaf': 20.0,
  'min_sum_hessian_in_leaf': 100.0,
  'num_leaves': 200.0,
  'subsample': 0.01})

In [25]:
with open('opt_params.pickle', 'wb') as f:
    pickle.dump(opt_params, f)